# Generating answers with source citations
### This Notebook contains a sample showing how to generate answers with inline & end of the answer citations pointing to the original source document used to answer the question
<div class="alert alert-block alert-warning">
<h4><u>Prerequisite:</u> Please run the <a href="../day5.ipynb" >Day 5 notebook</a> to create & populate the vector database before executing this notebook</h4>
</div>

 

In [0]:
# imports
import os
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from IPython.display import Markdown, display, update_display

In [0]:
load_dotenv()
MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [0]:
openai = OpenAI()
embeddings = OpenAIEmbeddings()

In [0]:
# Load the existing vector database that you created from the Day5 notebook
if os.path.exists(f"..\\{db_name}"):
  vectorstore = Chroma(embedding_function=embeddings, persist_directory=f"..\\{db_name}")
  print(f"Vectorstore loaded with {vectorstore._collection.count()} documents")
else:
  print("Vector store doesn't exist. Please run the Day 5 notebook first to create Chroma Vector DB & injest the data.")

In [0]:
system_message = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Use the following markdown format to answer the question along with the Source used to generate the answer, add inline citation for each sentence & add end of the answer citations:
'CEO of Insurellm is Avery Lancaster [[1]](Source Link 1). Who is also a co-founder [[2]](Source Link 2)
Citations: (Note: No duplicates allowed in the below list)

[1 - Source Title 1](Link 1)
[2 - Source Title 2](Link 2)
...
[n - Source Title n](Link n)'
 
Example answer: 
'CEO of Insurellm is Avery Lancaster [[1]](knowledge-base\\company\\about.md). Who is also a co-founder [[2]](knowledge-base\\employees\\Avery Lancaster.md)
Citations:

[1 - About Company](knowledge-base\\company\\about.md)
[2 - Avery Lancaster employees](knowledge-base\\employees\\Avery Lancaster.md)'
 
Important Note: Have unique end of the answer citations. Don't give duplicate citation numbers for the same source link, reuse the same citation number if the same source link is referenced multiple times.
'"""

In [0]:
# Change the below port if jupyter notebook is running in a different port
jupyter_notebook_port = "8888"

In [0]:
def generate_user_prompt(message):
    retriever = vectorstore.as_retriever()
    results = retriever.invoke(message)
    doc_chunk_merged = ""
    for doc_chunk in results:        
        source = f"http://localhost:{jupyter_notebook_port}/lab/tree/week5/" + doc_chunk.metadata.get("source").replace("\\","/")
        title = doc_chunk.metadata.get("doc_type") + " -> " + source.split('\\')[-1][:-3]
        doc_chunk_merged += f"Content: {doc_chunk.page_content}\n Source title: {title}\n Source link: {source}\n\n"
    return f"Question: {message}\n {doc_chunk_merged}"

In [0]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": generate_user_prompt(message)}]
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True, seed=3, max_tokens=1000)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [0]:
#Testing the Answer generation - 1
user_prompt = "Please explain what Insurellm is in a couple of sentences"

display_handle = display(Markdown(""), display_id=True)
for chunk in chat(user_prompt, []):
    update_display(Markdown(chunk), display_id=display_handle.display_id)

In [0]:
#Testing the Answer generation - 2
user_prompt = "Please explain in short on what products are available in Insurellm"

display_handle = display(Markdown(""), display_id=True)
for chunk in chat(user_prompt, []):
    update_display(Markdown(chunk), display_id=display_handle.display_id)

In [0]:
#Launch Gradio
gr.ChatInterface(fn=chat, type="messages").launch()